![](attachment:./RackMultipart20220329-4-1ky0ca6_html_ab3a6516da80d7cc.jpg)

# Relatórios

Agora, chegamos no momento em que a qualidade foi aferida e as transformações necessárias para que osdados sejam úteis, já foram realizadas. Este momento conseguimos responder perguntas importantes paradirecionar esforços para melhorar a qualidade dos dados e também responder perguntas de negócio quesão essenciais para tomada de decisão.

In [2]:
!pip install pyspark
!pip install findspark

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 5.0 MB 33.7 MB/s            
     |████████████████████████████████| 67 kB 4.3 MB/s             
     |████████████████████████████████| 812 kB 63.7 MB/s            
     |████████████████████████████████| 3.6 MB 63.5 MB/s            
     |████████████████████████████████| 133 kB 64.5 MB/s            
     |████████████████████████████████| 109 kB 67.5 MB/s            
     |████████████████████████████████| 55 kB 7.3 MB/s             
     |████████████████████████████████| 570 kB 56.7 MB/s            
     |████████████████████████████████| 74 kB 5.2 MB/s             
     |████████████████████████████████| 339 kB 58.6 MB/s            
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
  

In [3]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType

from datetime import datetime as dt

In [4]:
sc = SparkContext()

spark = (SparkSession.builder.master('local[2]').appName("Relatórios - wesley"))

In [5]:
"""

!pip install great_expectations
!printf 'yes' | great_expectations init


from ruamel import yaml

import great_expectations as ge
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
context = ge.get_context()
"""

In [5]:
# Expressoes regulares comuns
REGEX_ALPHA    = r'[a-zA-Z]+'
REGEX_INTEGER  = r'[0-9]+'
REGEX_FLOAT    = r'[0-9]+\.[0-9]+'
REGEX_ALPHANUM = r'[0-9a-zA-Z]+'
REGEX_EMPTY_STR= r'[\t ]+$'
REGEX_SPECIAL  = r'[!@#$%&*\(\)_]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [6]:
import re

def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) == '') | (F.col(col).rlike(REGEX_EMPTY_STR) ))

In [7]:
def add_test_rows_for_airports(data, _format):
    """ Adiciona linhas para testar regras de negócio"""
    
    values = [
        #faa    name  lat       lon               alt       tz    dst
        (None , None, None    , None            , None    , None, None),
        (''   , ''  , ''      , ''              , ''      , ''  , ''  ),
        ('   ', None, '12O.12', '-80.Aa6195833' , '1044Aa', '-2x', '34'),
        ('AAA', None, '12O.12', '-80.Aa6195833' , '1044Aa', '-2x', '34'),
        ('222', None, None    , '-80.Aa6195833' , '-100'  , '-14', 'K'),
        ('__!', None, None    , '-80.Aa6195833' , '-100'  , '-14', 'K')
    ]
 
    if _format == "df":
        return spark.getOrCreate().createDataFrame(values, data.columns).union(data)
    elif _format == "rdd":
        return sc.parallelize(tuple(values)).union(data)

def add_test_rows_for_flights(data, _format):
    """ Adiciona linhas para testar regras de negócio"""
    
    values = [
        #|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
        (None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None),
        (12,None,None,None,12,None,None,None,12,None,None,None,12,None,None,12,12),
        (13,12,1,123,None,12,None,12,None,12,None,12,None,12,None,12,22),
        (1950,13,29,2244,None,None,12,None,None,None,12,None,None,None,12,None,None),
        (1950,2,29,234,None,None,12,None,None,None,12,None,None,None,12,None,None),
        (2022,8,27,1345,None,None,12,None,None,None,12,None,None,None,12,None,None),
    ]
 
    if _format == "df":
        return spark.getOrCreate().createDataFrame(values, data.columns).union(data)
    elif _format == "rdd":
        return sc.parallelize(tuple(values)).union(data)

In [8]:
# Lendo arquivo parquet

flights_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/qa_flights.parquet"))


# Lendo arquivo parquet
planes_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/planes_qa.parquet"))

# Lendo arquivo parquet
airports_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/airport_qa.parquet"))

In [9]:
# Creating temp views to use with sql
flights_qa.createOrReplaceTempView('flights')
planes_qa.createOrReplaceTempView('planes')
airports_qa.createOrReplaceTempView('airports')

In [8]:
planes_qa.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|qa_year|qa_type| qa_manufacturer|qa_model|qa_engines|qa_seats|qa_speed|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1998|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1999|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

In [19]:
flights_qa.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_year_month_day|qa_hour_minute|qa_dep_arr_time|qa_dep_arr_delay|qa_carrier|qa_flight|qa_origin_dest|qa_air_time|qa_distance|qa_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|             null|          null|           null|            null|        VX|     1780|          n

In [23]:
airports_qa.show()

+-------+--------------------+---------+-----------+-------+------+-------+----------+--------------------+----------+-----------+----------+---------+----------+
|air_faa|            air_name|  air_lat|    air_lon|air_alt|air_tz|air_dst|air_qa_faa|         air_qa_name|air_qa_lat| air_qa_lon|air_qa_alt|air_qa_tz|air_qa_dst|
+-------+--------------------+---------+-----------+-------+------+-------+----------+--------------------+----------+-----------+----------+---------+----------+
|    04G|   Lansdowne Airport|41.130474|  -80.61958|   1044|  -5.0|      A|       04G|   Lansdowne Airport| 41.130474|  -80.61958|      1044|     -5.0|         A|
|    06A|Moton Field Munic...| 32.46057|  -85.68003|    264|  -5.0|      A|       06A|Moton Field Munic...|  32.46057|  -85.68003|       264|     -5.0|         A|
|    06C| Schaumburg Regional| 41.98934|  -88.10124|    801|  -6.0|      A|       06C| Schaumburg Regional|  41.98934|  -88.10124|       801|     -6.0|         A|
|    06N|     Randall 

## Pergunta 1:

In [10]:
# Renomeando colunas
flights_qa = flights_qa.select(*(F.col(x).alias('fl_' + x ) for x in flights_qa.columns))
planes_qa = planes_qa.select(*(F.col(x).alias('pl_' + x ) for x in planes_qa.columns))
airports_qa2 = airports_qa.select(*(F.col(x).alias('air2_' + x ) for x in airports_qa.columns))
airports_qa = airports_qa.select(*(F.col(x).alias('air_' + x ) for x in airports_qa.columns))

In [11]:
flights_qa.show()

+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-------------+
|fl_year|fl_month|fl_day|fl_dep_time|fl_dep_delay|fl_arr_time|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_destiny|fl_air_time|fl_distance|fl_hour|fl_minute|fl_qa_year_month_day|fl_qa_hour_minute|fl_qa_dep_arr_time|fl_qa_dep_arr_delay|fl_qa_carrier|fl_qa_flight|fl_qa_origin_dest|fl_qa_air_time|fl_qa_distance|fl_qa_airtime|
+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-

In [13]:
qa_challenge_df = flights_qa.join(airports_qa,  (flights_qa.fl_origin  == airports_qa.air_faa) , 'left')\
                            .join(airports_qa2, (flights_qa.fl_destiny == airports_qa2.air2_faa),'left')\
                            .join(planes_qa,    (flights_qa.fl_tailnum == planes_qa.pl_tailnum), 'left')

In [14]:
qa_challenge_df.show()

+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-------------+-------+-------------------+---------+---------+-------+------+-------+----------+-------------------+----------+----------+----------+---------+----------+--------+--------------------+---------+-----------+--------+-------+--------+-----------+--------------------+-----------+-----------+-----------+----------+-----------+----------+-------+--------------------+---------------+-----------+----------+--------+--------+---------+-------------+----------+----------+------------------+-----------+-------------+-----------+-----------+------------+
|fl_year|fl_month|fl_day|fl_dep_time|fl_dep_delay|fl_arr_time|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_o

In [18]:
#joined_dfs.select([joined_dfs.groupBy(F.col(c).startswith(c).alias(c)).count().show() for c in joined_dfs.columns if 'qa' in c]).show()

+------+-----+
|qa_faa|count|
+------+-----+
| false|10001|
|  null| 4024|
+------+-----+

+-------+-----+
|qa_name|count|
+-------+-----+
|   null|14025|
+-------+-----+

+------+-----+
|qa_lat|count|
+------+-----+
| false|14025|
+------+-----+

+------+-----+
|qa_lon|count|
+------+-----+
| false| 4179|
|  null| 9846|
+------+-----+

+------+-----+
|qa_alt|count|
+------+-----+
| false|14025|
+------+-----+

+-----+-----+
|qa_tz|count|
+-----+-----+
|false|14025|
+-----+-----+

+------+-----+
|qa_dst|count|
+------+-----+
| false|12553|
|  null| 1472|
+------+-----+



TypeError: TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

## Pergunta 2:

In [18]:
# print(all_cols_join.columns)
# all_cols_join.select('*').show()
# for c in all_cols_join.columns:
# all_cols_join.select(F.count(F.when(F.col(c) == 'M', 1)) ).show()

for i in ['M', 'F', 'I', 'S', 'T']:
    print(i)
    joined_dfs.select([F.count(F.when(F.col(c) == i, 1)).alias(c) for c in joined_dfs.columns]).show(vertical=True)
print('-'*200)
for i in ['M', 'F', 'I', 'S', 'T']:
    print(i)
    transformation_c_dfs.select([F.count(F.when(F.col(c) == i, 1)).alias(c) for c in transformation_c_dfs.columns]).show(vertical=True)

M
-RECORD 0-------
 faa     | 0    
 name    | 0    
 lat     | 0    
 lon     | 0    
 alt     | 0    
 tz      | 0    
 dst     | 0    
 qa_faa  | 0    
 qa_name | 0    
 qa_lat  | 2628 
 qa_lon  | 2676 
 qa_alt  | 2675 
 qa_tz   | 55   
 qa_dst  | 0    

F
-RECORD 0--------
 faa     | 0     
 name    | 0     
 lat     | 0     
 lon     | 0     
 alt     | 0     
 tz      | 0     
 dst     | 0     
 qa_faa  | 10001 
 qa_name | 0     
 qa_lat  | 0     
 qa_lon  | 0     
 qa_alt  | 0     
 qa_tz   | 0     
 qa_dst  | 0     

I
-RECORD 0--------
 faa     | 0     
 name    | 0     
 lat     | 0     
 lon     | 0     
 alt     | 0     
 tz      | 0     
 dst     | 0     
 qa_faa  | 0     
 qa_name | 0     
 qa_lat  | 11397 
 qa_lon  | 1503  
 qa_alt  | 5896  
 qa_tz   | 4083  
 qa_dst  | 0     

S
-RECORD 0------
 faa     | 0   
 name    | 0   
 lat     | 0   
 lon     | 0   
 alt     | 0   
 tz      | 0   
 dst     | 0   
 qa_faa  | 0   
 qa_name | 0   
 qa_lat  | 0   
 qa_lon  | 0   
 q

In [20]:
# Com List Comprehension
qa_challenge_df.select([qa_challenge_df.groupBy(F.col(c).startswith(c).alias(c)).count().show() for c in qa_challenge_df.columns if 'qa' in c])

+--------------------+-----+
|fl_qa_year_month_day|count|
+--------------------+-----+
|                null|10000|
+--------------------+-----+

+-----------------+-----+
|fl_qa_hour_minute|count|
+-----------------+-----+
|            false|   48|
|             null| 9952|
+-----------------+-----+

+------------------+-----+
|fl_qa_dep_arr_time|count|
+------------------+-----+
|             false|   55|
|              null| 9945|
+------------------+-----+

+-------------------+-----+
|fl_qa_dep_arr_delay|count|
+-------------------+-----+
|              false|   75|
|               null| 9925|
+-------------------+-----+

+-------------+-----+
|fl_qa_carrier|count|
+-------------+-----+
|        false|10000|
+-------------+-----+

+------------+-----+
|fl_qa_flight|count|
+------------+-----+
|       false|10000|
+------------+-----+

+-----------------+-----+
|fl_qa_origin_dest|count|
+-----------------+-----+
|             null|10000|
+-----------------+-----+

+--------------+-

TypeError: TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [133]:
i, most_M = float('-inf'), ''
results = []

for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter(~check_empty_column(c) & (F.col(c) =='M')).groupBy(F.substring(c, 1, 1).alias(c)).count().toPandas())

for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_M = c

print(most_M, ' -> ' ,i)

pl_qa_type  ->  9444


In [135]:
i, most_F = float('-inf'), ''
results = []
for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter((F.col(c)!='') & (F.col(c) =='F')).groupBy(F.substring(c, 1, 1).alias(c)).count().toPandas())

for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_F = c

print(most_F, ' -> ' ,i)

air_qa_faa  ->  10000


In [136]:
results = []
## 5. Qual variável apresenta maior número de formatos errados?
for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter((F.col(c)!='') & (F.col(c) =='I')) \
               .groupBy(F.substring(c, 1, 1).alias(c))\
               .count()\
               .toPandas())

In [137]:
i, most_I = float('-inf'), '' 
for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_I = c

print(most_I, ' -> ' ,i)

pl_qa_year  ->  8


In [23]:
# Lendo arquivo parquet

flights_transform = (spark.getOrCreate().read
                          .format("parquet")
                          .option("header","True")
                          .load("./parquet_files/transformation_proc/flights_proc.parquet"))


# Lendo arquivo parquet
planes_transform = (spark.getOrCreate().read
                         .format("parquet")
                         .option("header","True")
                         .load("./parquet_files/transformation_proc/planes_proc.parquet"))

# Lendo arquivo parquet
airports_transform = (spark.getOrCreate().read
                           .format("parquet")
                           .option("header","True")
                           .load("./parquet_files/transformation_proc/airports_proc.parquet"))

In [24]:
# Renomeando colunas
flights_transform   = flights_transform.select(*(F.col(x).alias('fl_' + x ) for x in flights_transform.columns))
planes_transform    = planes_transform.select(*(F.col(x).alias('pl_' + x ) for x in planes_transform.columns))
airports_transform2 = airports_transform.select(*(F.col(x).alias('air2_' + x ) for x in airports_transform.columns))
airports_transform  = airports_transform.select(*(F.col(x).alias('air_' + x ) for x in airports_transform.columns))

In [25]:
flights_transform.show()

+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+
|fl_dep_time|fl_arr_time|fl_dep_delay|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_dest|fl_air_time|fl_distance|    fl_dep_datetime|fl_air_time_projected|fl_air_time_expected|fl_haul_duration|fl_dep_season|fl_dep_delay_category|
+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+
|        658|        935|          -7|          -5|        VX|    N846VA|     1780|      SEA|    LAX|        132|        954|2014-12-08 08:58:00|                  115|                 126|      SHORT-HAUL|         FALL|          ANTECIPATED|
|       1040|       1505|       

In [26]:
transform_proc_df = flights_transform.join(airports_transform,  (flights_transform.fl_origin  == airports_transform.air_faa) , 'left')\
                                     .join(airports_transform2, (flights_transform.fl_dest    == airports_transform2.air2_faa),'left')\
                                     .join(planes_transform,    (flights_transform.fl_tailnum == planes_transform.pl_tailnum), 'left')

In [27]:
transform_proc_df.show()

+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+-------+-------------------+---------+---------+-------+------+-------+-------------+--------+------------+------------------+--------+--------------------+---------+-----------+--------+-------+--------+-------------+---------+-------------+-------------------+----------+-------+---------+---------------+-----------+----------+--------+--------+---------+-----------+------+--------------+
|fl_dep_time|fl_arr_time|fl_dep_delay|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_dest|fl_air_time|fl_distance|    fl_dep_datetime|fl_air_time_projected|fl_air_time_expected|fl_haul_duration|fl_dep_season|fl_dep_delay_category|air_faa|           air_name|  air_lat|  air_lon|air_alt|air_tz|air_dst|   air_region|air_type|air_military|air_administration

In [28]:
#Regiões de partida  - 
# transform_proc_df.select(F.countDistinct('air_name')).groupBy('air_region').orderBy('region').show() | #.agg(F.expr('count(distinct air_name)').alias("Aeroportos por regiao")).show()

# Regiões de chegada -
transform_proc_df.groupBy(F.col('air2_region').alias('Regiões'))\
                 .agg(F.expr('count(distinct air2_name)').alias('Aeroportos por região'))\
                 .show()

+-------------+---------------------+
|      regiões|aeroportos por região|
+-------------+---------------------+
|       ALASKA|                    9|
|MAINLAND-EAST|                   24|
|MAINLAND-WEST|                   36|
+-------------+---------------------+



## Pergunta 4:

In [29]:
transform_proc_df.agg(F.max(F.col('air_alt')-F.col('air2_alt')).alias("Maior diferença de altiude (em pés)")).show()  

+-----------------------------------+
|Maior diferença de altiude (em pés)|
+-----------------------------------+
|                                429|
+-----------------------------------+



## Pergunta 5:

In [114]:
transform_proc_df.filter(transform_proc_df['fl_arr_delay'] > 0)\
                 .agg(F.format_number(F.avg('fl_arr_delay'), 1)\
                 .alias('Atraso médio das chegadas (em minutos)'))\
                .show()
#--------
transform_proc_df.filter(transform_proc_df['fl_dep_delay'] > 0)\
                 .agg(F.format_number(F.avg('fl_dep_delay'), 1)\
                 .alias('Atraso médio de partida (em minutos)'))\
                .show()

+--------------------------------------+
|Atraso médio das chegadas (em minutos)|
+--------------------------------------+
|                                  24.6|
+--------------------------------------+

+------------------------------------+
|Atraso médio de partida (em minutos)|
+------------------------------------+
|                                25.5|
+------------------------------------+



## Pergunta 7:

In [116]:
transform_proc_df.filter(transform_proc_df['fl_arr_delay'] > 0)\
                 .groupBy(F.col('air2_region')\
                    .alias('regiões'))\
                 .agg(F.ceil(F.avg('fl_arr_delay'))\
                    .alias('Atraso médio de chegada (em minutos)'))\
                .show()
#------
transform_proc_df.filter(transform_proc_df['fl_dep_delay'] > 0)\
                 .groupBy(F.col('air2_region')\
                    .alias('regiões'))\
                 .agg(F.ceil(F.avg('fl_dep_delay'))\
                    .alias('Atraso médio de partida (em minutos)'))\
                .show()

+-------------+------------------------------------+
|      regiões|Atraso médio de chegada (em minutos)|
+-------------+------------------------------------+
|       ALASKA|                                  23|
|MAINLAND-EAST|                                  29|
|MAINLAND-WEST|                                  24|
+-------------+------------------------------------+

+-------------+------------------------------------+
|      regiões|Atraso médio de partida (em minutos)|
+-------------+------------------------------------+
|       ALASKA|                                  21|
|MAINLAND-EAST|                                  27|
|MAINLAND-WEST|                                  26|
+-------------+------------------------------------+



## Pergunta 8:

In [117]:
#transform_proc_df.filter(transform_proc_df['fl_arr_delay'] > 0).groupBy(F.col('air_region').alias('regiões')).agg(F.ceil(F.avg('fl_arr_delay')).alias('Atraso médio de chegada (em minutos)')).show()
transform_proc_df.filter(transform_proc_df['fl_dep_delay'] > 0).groupBy(F.col('air_region').alias('regiões')).agg(F.ceil(F.avg('fl_dep_delay')).alias('Atraso médio de partida (em minutos)')).show()

+-------------+------------------------------------+
|      regiões|Atraso médio de partida (em minutos)|
+-------------+------------------------------------+
|MAINLAND-WEST|                                  26|
+-------------+------------------------------------+



## Pergunta 9:

In [120]:
transform_proc_df.filter((transform_proc_df['fl_arr_delay'] > 0) | (transform_proc_df['fl_dep_delay'] > 0))\
                 .groupBy(F.year(F.to_timestamp('fl_dep_datetime'))\
                    .alias('Ano'))\
                 .agg((F.sum('fl_dep_delay') + F.sum('fl_arr_delay'))\
                    .alias('Atraso de partida e chegada acumulados (em minutos)'))\
                .show()

+----+---------------------------------------------------+
| Ano|Atraso de partida e chegada acumulados (em minutos)|
+----+---------------------------------------------------+
|2015|                                                  6|
|2014|                                             166792|
+----+---------------------------------------------------+



*Abaixo seguem os atrasasos acumulados por partida e por chegada.*

In [124]:
transform_proc_df.filter(F.col('fl_dep_delay')>0)\
                 .groupBy(F.year(F.to_timestamp('fl_dep_datetime'))\
                    .alias('Ano'))\
                 .agg(F.sum('fl_dep_delay')\
                    .alias('Atraso de partida acumulado (em minutos)'))\
                .show()

transform_proc_df.filter(F.col('fl_arr_delay')>0)\
                 .groupBy(F.year(F.to_timestamp('fl_dep_datetime'))\
                    .alias('Ano'))\
                 .agg(F.sum('fl_arr_delay')\
                    .alias('Atraso de chegada acumulado (em minutos)'))\
                .show()

+----+----------------------------------------+
| Ano|Atraso de partida acumulado (em minutos)|
+----+----------------------------------------+
|2014|                                   88314|
+----+----------------------------------------+

+----+----------------------------------------+
| Ano|Atraso de chegada acumulado (em minutos)|
+----+----------------------------------------+
|2015|                                       9|
|2014|                                   91811|
+----+----------------------------------------+



## Pergunta 10:

In [131]:
transform_proc_df.filter((F.col('fl_arr_delay')>0) |( F.col('fl_dep_delay') > 0 ))\
                 .groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano'), F.col('air2_region').alias('Região'))\
                 .agg((F.sum('fl_dep_delay') + F.sum('fl_arr_delay'))\
                        .alias('Atraso destino acumulado (em minutos)'))\
                .show()

+----+-------------+-------------------------------------+
| Ano|       Região|Atraso destino acumulado (em minutos)|
+----+-------------+-------------------------------------+
|2014|       ALASKA|                                14281|
|2014|MAINLAND-EAST|                                40558|
|2015|MAINLAND-WEST|                                    6|
|2014|MAINLAND-WEST|                               111953|
+----+-------------+-------------------------------------+



## Pergunta 11:

In [38]:
transform_proc_df.agg(F.ceil(F.avg('fl_air_time')).alias('Tempo médio de vôo (em minutos)')).show()

+-------------------------------+
|Tempo médio de vôo (em minutos)|
+-------------------------------+
|                            153|
+-------------------------------+



## Pergunta 12:

In [132]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões'))\
                 .agg(F.ceil(F.avg('fl_air_time'))\
                    .alias('Tempo médio de vôo (em minutos)'))\
                .show()

+-------------+-------------------------------+
|      regiões|Tempo médio de vôo (em minutos)|
+-------------+-------------------------------+
|       ALASKA|                            228|
|MAINLAND-EAST|                            238|
|MAINLAND-WEST|                            116|
+-------------+-------------------------------+



In [40]:
transform_proc_df.groupBy(F.col('fl_origin').alias('Origem'), F.col('fl_dest').alias('Destino'))\
                 .agg(F.ceil(F.avg('fl_air_time'))\
                    .alias('Tempo médio de vôo (em minutos)'))\
                .show(200)

+------+-------+-------------------------------+
|Origem|Destino|Tempo médio de vôo (em minutos)|
+------+-------+-------------------------------+
|   SEA|    RNO|                             75|
|   SEA|    DTW|                            220|
|   SEA|    CLE|                            234|
|   SEA|    LAX|                            127|
|   PDX|    SEA|                             35|
|   SEA|    BLI|                             23|
|   PDX|    IAH|                            214|
|   PDX|    PHX|                            131|
|   SEA|    SLC|                             89|
|   SEA|    SBA|                            119|
|   SEA|    BWI|                            270|
|   PDX|    IAD|                            268|
|   PDX|    SFO|                             86|
|   SEA|    KOA|                            348|
|   PDX|    MCI|                            175|
|   SEA|    SJC|                            104|
|   SEA|    ABQ|                            143|
|   SEA|    SAT|    

In [41]:
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano'))\
                 .agg(F.ceil(F.avg('fl_air_time'))\
                    .alias('Tempo médio de vôo (em minutos)'))\
                .show()

+----+-------------------------------+
| Ano|Tempo médio de vôo (em minutos)|
+----+-------------------------------+
|2015|                             43|
|2014|                            153|
+----+-------------------------------+



In [42]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões'))\
                 .agg(F.sum('fl_air_time')\
                    .alias('Tempo de vôo acumulado (em minutos)'))\
                .show()

+-------------+-----------------------------------+
|      regiões|Tempo de vôo acumulado (em minutos)|
+-------------+-----------------------------------+
|       ALASKA|                             230602|
|MAINLAND-EAST|                             508344|
|MAINLAND-WEST|                             789679|
+-------------+-----------------------------------+



In [43]:
transform_proc_df.agg(F.round(F.avg('fl_distance'))
                    .alias('Distância de vôo médio (em milhas)'))\
                .show()

+----------------------------------+
|Distância de vôo médio (em milhas)|
+----------------------------------+
|                            1208.0|
+----------------------------------+



*Considerando que a distância dos vôos é dada pela distância total entre os aeroportos*:

In [44]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.round(F.avg('fl_distance')).alias('Distância de vôo médio (em milhas)')).show()

+-------------+----------------------------------+
|      regiões|Distância de vôo médio (em milhas)|
+-------------+----------------------------------+
|       ALASKA|                            1742.0|
|MAINLAND-EAST|                            2042.0|
|MAINLAND-WEST|                             868.0|
+-------------+----------------------------------+



In [45]:
transform_proc_df.groupBy(F.col('fl_origin').alias('Origem'), F.col('fl_dest').alias('Destino')).agg(F.concat(F.round(F.avg('fl_distance')), F.lit(' mls')).alias('Distância de vôo médio (em milhas)')).show()

+------+-------+----------------------------------+
|Origem|Destino|Distância de vôo médio (em milhas)|
+------+-------+----------------------------------+
|   SEA|    RNO|                         564.0 mls|
|   SEA|    DTW|                        1927.0 mls|
|   SEA|    CLE|                        2021.0 mls|
|   SEA|    LAX|                         954.0 mls|
|   PDX|    SEA|                         129.0 mls|
|   SEA|    BLI|                          93.0 mls|
|   PDX|    IAH|                        1825.0 mls|
|   PDX|    PHX|                        1009.0 mls|
|   SEA|    SLC|                         689.0 mls|
|   SEA|    SBA|                         908.0 mls|
|   SEA|    BWI|                        2335.0 mls|
|   PDX|    IAD|                        2327.0 mls|
|   PDX|    SFO|                         550.0 mls|
|   SEA|    KOA|                        2688.0 mls|
|   PDX|    MCI|                        1482.0 mls|
|   SEA|    SJC|                         697.0 mls|
|   SEA|    

In [46]:
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano')).agg(F.concat(F.format_number(F.sum('fl_distance'), 1), F.lit('mls')).alias('Distância de vôo acumulado (em milhas)')).show()
#.agg(F.format_number(F.sum('fl_distance'), 1).alias('Distância de vôo acumulado (em milhas)')).show()

+----+--------------------------------------+
| Ano|Distância de vôo acumulado (em milhas)|
+----+--------------------------------------+
|2015|                              224.0mls|
|2014|                       12,081,292.0mls|
+----+--------------------------------------+



In [47]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.concat(F.format_number(F.sum('fl_distance'), 1), F.lit('mls')).alias('Distância de vôo acumulado (em milhas)')).show()

+-------------+--------------------------------------+
|      regiões|Distância de vôo acumulado (em milhas)|
+-------------+--------------------------------------+
|       ALASKA|                        1,762,553.0mls|
|MAINLAND-EAST|                        4,378,902.0mls|
|MAINLAND-WEST|                        5,940,061.0mls|
+-------------+--------------------------------------+



*Considerando que todos os aviões dos vôos estão em sua capacidade máxima de assentos, temos os seguintes resultados*: 

In [48]:
transform_proc_df.groupBy(F.col('fl_origin').alias('Origem'), F.col('fl_dest').alias('Destino') ).agg(F.concat(F.format_number(F.ceil(F.avg('pl_seats')), 0), F.lit(' passageiros')).alias('Número médio de passageiros por rota')).show()

+------+-------+------------------------------------+
|Origem|Destino|Número médio de passageiros por rota|
+------+-------+------------------------------------+
|   SEA|    RNO|                     142 passageiros|
|   SEA|    DTW|                     213 passageiros|
|   SEA|    CLE|                     182 passageiros|
|   SEA|    LAX|                     155 passageiros|
|   PDX|    SEA|                      65 passageiros|
|   SEA|    BLI|                     164 passageiros|
|   PDX|    IAH|                     183 passageiros|
|   PDX|    PHX|                     196 passageiros|
|   SEA|    SLC|                     166 passageiros|
|   SEA|    SBA|                      80 passageiros|
|   SEA|    BWI|                     152 passageiros|
|   PDX|    IAD|                     188 passageiros|
|   PDX|    SFO|                     139 passageiros|
|   SEA|    KOA|                     171 passageiros|
|   PDX|    MCI|                     147 passageiros|
|   SEA|    SJC|            

In [49]:
transform_proc_df.groupBy(F.year(F.to_timestamp('fl_dep_datetime')).alias('Ano')).agg(F.format_number(F.ceil(F.sum('pl_seats')), 0).alias('Número acumulado de passageiros por ano')).show()

+----+---------------------------------------+
| Ano|Número acumulado de passageiros por ano|
+----+---------------------------------------+
|2015|                                    149|
|2014|                              1,509,395|
+----+---------------------------------------+



In [50]:
transform_proc_df.groupBy(F.col('fl_dest').alias('Destino')).count().orderBy(F.col('count').desc()).withColumnRenamed('count', 'Quantidade de viagens').show(1)

+-------+---------------------+
|Destino|Quantidade de viagens|
+-------+---------------------+
|    SFO|                  787|
+-------+---------------------+
only showing top 1 row



*Considerando que a quantidade de assentos indica a quantidade de passageiros num determinado destino*:

In [51]:
transform_proc_df.groupBy(F.col('fl_dest').alias('Destino')).agg(F.sum('pl_seats').alias('sum_seats')).orderBy(F.col('sum_seats').desc()).withColumnRenamed('sum_seats', 'Quantidade de passageiros').show(1)

+-------+-------------------------+
|Destino|Quantidade de passageiros|
+-------+-------------------------+
|    SFO|                   119635|
+-------+-------------------------+
only showing top 1 row



*Considerando a distância da rota para **PDX** ou partindo de **PDX**, temos que o trajeto mais distante descrito abaixo:*

In [52]:
transform_proc_df.select(F.col('fl_origin').alias('Origem'),F.col('fl_dest').alias('Destino'),F.concat(F.format_number(F.col('fl_distance'), 1), F.lit(' mls')).alias('Distância em milhas')).filter((F.col('fl_origin') == 'PDX')| (F.col('fl_dest') == 'PDX')).orderBy(F.col('fl_distance').desc()).show(1)

+------+-------+-------------------+
|Origem|Destino|Distância em milhas|
+------+-------+-------------------+
|   PDX|    LIH|        2,631.0 mls|
+------+-------+-------------------+
only showing top 1 row



In [53]:
transform_proc_df.groupBy(F.col('fl_dest').alias('Destino')).count().orderBy(F.col('count').desc()).withColumnRenamed('count', 'Total de viagens').show(1)

+-------+----------------+
|Destino|Total de viagens|
+-------+----------------+
|    SFO|             787|
+-------+----------------+
only showing top 1 row



*Considerando a quantidade de vezes utilizadas nas viagens, como indicador de popularidade:*

In [54]:
transform_proc_df.select('*').groupBy(F.col('pl_model').alias('Modelo')).count().orderBy(F.col('count').desc()).withColumnRenamed('count', 'Quantidade').show(1)

+-------+----------+
| Modelo|Quantidade|
+-------+----------+
|737-890|      1463|
+-------+----------+
only showing top 1 row



In [55]:
transform_proc_df.filter(F.col('pl_model').isNotNull()).groupBy(F.col('fl_dest').alias('Destino'), F.col('pl_model').alias('Modelo')).count().orderBy(F.col('count').desc()).withColumnRenamed('count', 'Quantidade').show(1)

+-------+-------+----------+
|Destino| Modelo|Quantidade|
+-------+-------+----------+
|    OAK|737-7H4|       141|
+-------+-------+----------+
only showing top 1 row



In [58]:
transform_proc_df.groupBy(F.col('fl_haul_duration').alias('Haul Duration')).agg(F.ceil(F.avg('pl_engines')).alias('Número médio de motores')).show()

+-------------+-----------------------+
|Haul Duration|Número médio de motores|
+-------------+-----------------------+
|    LONG-HAUL|                      2|
|  MEDIUM-HAUL|                      2|
|   SHORT-HAUL|                      2|
+-------------+-----------------------+



In [60]:
transform_proc_df.groupBy(F.col('fl_dep_season').alias('Estação do ano')).count().orderBy(F.col('count').desc()).withColumnRenamed('count', 'Quantidade de vôos').show(1) 

+--------------+------------------+
|Estação do ano|Quantidade de vôos|
+--------------+------------------+
|        SUMMER|              2918|
+--------------+------------------+
only showing top 1 row



*Considerando a quantidade acumulada de vôos em cada um dos destinos com suas respectivas estações do ano, como indicador para a Estação do ano mais popular em cada destino, temos:* 

In [81]:
data = transform_proc_df.groupBy(F.col('fl_dep_season'), F.col('fl_dest')).count().orderBy(F.col('count').desc())

In [86]:
maxs =  data.groupBy(F.col('fl_dest').alias('max_fl_dest')).agg(F.max('count').alias('max')).alias('maxs')

result = data.join(maxs,
             F.col('count') == F.col('max') 
         ).select('*')

In [96]:
result.select(F.col('fl_dep_season').alias('Estação do Ano'), F.col('fl_dest').alias('Destino'), F.col('count').alias('Quantidade')).orderBy(F.col('max').desc()).show()

+--------------+-------+----------+
|Estação do Ano|Destino|Quantidade|
+--------------+-------+----------+
|        SUMMER|    SFO|       217|
|        SPRING|    LAX|       176|
|        SUMMER|    DEN|       172|
|        SPRING|    PHX|       148|
|        SPRING|    LAS|       145|
|          FALL|    DEN|       145|
|        SPRING|    LAS|       145|
|        SUMMER|    ANC|       145|
|          FALL|    DEN|       145|
|        SUMMER|    ANC|       145|
|        SUMMER|    ORD|       129|
|        SUMMER|    DFW|       122|
|        SUMMER|    SJC|       120|
|        SPRING|    SLC|       107|
|        SUMMER|    SAN|       100|
|          FALL|    OAK|        96|
|        SUMMER|    SMF|        91|
|        SUMMER|    ATL|        86|
|        SPRING|    DFW|        86|
|          FALL|    MSP|        68|
+--------------+-------+----------+
only showing top 20 rows



*Considerando a quantidade acumulada de ocorrências como indicador para qual categoria de atraso é mais comum, temos:*

In [97]:

transform_proc_df.groupBy('fl_dep_delay_category')\
                 .count()\
                 .orderBy(F.col('count').desc())\        
                 .withColumnRenamed('count', 'Quantidade')\
                 .show(1)

+---------------------+-----+
|fl_dep_delay_category|count|
+---------------------+-----+
|          ANTECIPATED| 5894|
+---------------------+-----+
only showing top 1 row



*Considerando o máximo da quantidade ocorrências da categoria de atraso por cada rota, temos que:*

In [103]:
data = transform_proc_df.groupBy(F.col('fl_dep_delay_category'), F.col('fl_origin'), F.col('fl_dest'))\
                        .count()\
                        .orderBy(F.col('count').desc())

In [109]:
maxs =  data.groupBy(F.col('fl_origin').alias('max_fl_origin'), F.col('fl_dest').alias('max_fl_dest')).agg(F.max('count').alias('max')).alias('maxs')

results = data.join(maxs,data['count'] == maxs['max'], 'inner')

In [113]:

results.select(F.col('fl_dep_delay_category').alias('Categoria'), F.col('fl_origin').alias('Origem'), F.col('fl_dest').alias('Destino'), F.col('count').alias('Quantidade'))\
       .orderBy(F.col('count').desc())\
       .show()

+-----------+------+-------+----------+
|  Categoria|Origem|Destino|Quantidade|
+-----------+------+-------+----------+
|ANTECIPATED|   SEA|    LAX|       293|
|ANTECIPATED|   SEA|    SFO|       245|
|ANTECIPATED|   SEA|    LAS|       228|
|ANTECIPATED|   SEA|    PHX|       195|
|ANTECIPATED|   SEA|    ANC|       192|
|ANTECIPATED|   PDX|    SFO|       174|
|ANTECIPATED|   SEA|    DEN|       169|
|ANTECIPATED|   SEA|    ORD|       167|
|ANTECIPATED|   PDX|    PHX|       136|
|ANTECIPATED|   SEA|    SJC|       135|
|ANTECIPATED|   PDX|    DEN|       135|
|ANTECIPATED|   PDX|    DEN|       135|
|ANTECIPATED|   SEA|    SJC|       135|
|ANTECIPATED|   SEA|    DFW|       134|
|ANTECIPATED|   PDX|    LAX|       126|
|ANTECIPATED|   PDX|    LAX|       126|
|ANTECIPATED|   PDX|    LAX|       126|
|ANTECIPATED|   SEA|    SLC|       126|
|ANTECIPATED|   SEA|    SLC|       126|
|ANTECIPATED|   SEA|    PDX|       126|
+-----------+------+-------+----------+
only showing top 20 rows

